In [1]:
import unidecode
import pandas as pd
import numpy as np

In [2]:
class Draft:
    def __init__(self, 
                 player_names, 
                 current_standings_df, 
                 randomize_order=True):
        self.players_lst = [Player(name) for name in player_names]
        self.driver_pool = DriverPool(current_standings_df)
        self.current_idx = 0
        self.picking_order = [0, 1, 2, 3, 3, 2, 1, 0]
        if randomize_order:
            np.random.shuffle(self.players_lst)
        print('Welcome to the Draft!')
        print('---------------------')
        print('The Draft Order is:')
        for n, p in enumerate(self.players_lst, 1):
            print(f'\t{n}. {p.name}')
    
    def __repr__(self):
        print(f"{self.players_lst[self.picking_order[self.current_idx]].name} is on the clock!")
        print('*'*40)
        print(self.driver_pool)
        return ''
    
    def show_rosters(self):
        for p in self.players_lst:
            print(p)
    
    def move_idx(self):
        if self.current_idx == len(self.picking_order) - 1:
            self.current_idx = 0
        else:
            self.current_idx += 1
    
    def pick(self, last_name, force=False):
        driver = self.driver_pool.get_driver(last_name)
        if not driver:
            print(f'Driver: ``{last_name}`` not available!')
            return
        pick_num = sum([len(p.roster) for p in self.players_lst]) + 1
        pick = (self.players_lst[self.picking_order[self.current_idx]]
                .pick_driver(driver, pick_num, force))
        if not pick:
            return
        self.move_idx()
        
    def save_rosters(self):
        all_dfs = [player.get_roster_dataframe() for player in self.players_lst]
        df = pd.concat(all_dfs).reset_index(drop=True)
        cols = [
            'OverallPick',
            'TeamPick',
            'Team',
            'LastName',
            'FirstName',
            'Car',
            'Nationality',
        ]
        df = (
            df[cols]
            .sort_values('OverallPick')
            .reset_index(drop=True)
        )
        df.to_csv('fantasy_rosters.csv')
        return df


class Player:
    def __init__(self, name, budget=153):
        self.budget = budget
        self.name = name
        self.roster = []
        
    def pick_driver(self, driver, pick_num, force=False):
        if not force and driver.cost > self.budget:
            print(f'Cannot afford ``{driver.name}``. Budget: {self.budget}.')
            return
        self.budget -= driver.cost
        self.roster.append((pick_num, driver))
        print(f'{self.name} selected {driver.name}!')
        print(f'\t{self.budget} points remaining.')
        self.__repr__()
        return 1
        
    def __repr__(self):
        print(f'{self.name} | Budget: {self.budget}')
        for d in self.roster:
            print(f'\t{d}')
        return ''
    
    def get_roster_dataframe(self):
        rows = []
        for overall, d in self.roster:
            row = d.self_dict()
            row['OverallPick'] = overall
            rows.append(row)
        df = pd.DataFrame(rows)
        df['Team'] = self.name
        df['TeamPick'] = [1, 2, 3, 4, 5]
        return df
    
class RaceDriver:
    def __init__(self, name, nationality, car, cost):
        self.name = unidecode.unidecode(name)
        self.first, self.last = self.name.split()
        self.nationality = nationality
        self.car = car
        self.cost = cost
        
    def __str__(self):
        return f'{self.name} | {self.nationality} | {self.car} | {self.cost}'
    
    def __repr__(self):
        return f'{self.name} | {self.nationality} | {self.car} | {self.cost}'
    
    def self_dict(self):
        return dict(
            LastName=self.last,
            FirstName=self.first,
            Nationality=self.nationality,
            Car=self.car,
        )
    
    
class DriverPool:
    def __init__(self, df):
        drivers_lst = [
            RaceDriver(r['DRIVER'], r['NATIONALITY'], r['CAR'], int(r['PTS']))
            for idx, r in df.iterrows()
        ]
        self.pool = {d.last: d for d in drivers_lst}
        
    def __repr__(self):
        return '\n'.join([d.__str__() for d in self.pool.values()])
    
    def get_driver(self, last_name):
        return self.pool.pop(last_name, None)

In [3]:
current_standings = pd.read_csv('current_standings.csv', index_col=0)

In [4]:
TEAM_NAMES = [
    'Carlaniel & the Romeos',
    'Scuderia Spaghetti',
    '3 Orange Whips',
    'Deep Fried'
]

# Draft Start

In [5]:
main = Draft(TEAM_NAMES, 
             current_standings, 
             randomize_order=False)

Welcome to the Draft!
---------------------
The Draft Order is:
	1. Carlaniel & the Romeos
	2. Scuderia Spaghetti
	3. 3 Orange Whips
	4. Deep Fried


## R1

In [6]:
main

Carlaniel & the Romeos is on the clock!
****************************************
Max Verstappen | NED | RED BULL RACING HONDA | 105
Lewis Hamilton | GBR | MERCEDES | 101
Sergio Perez | MEX | RED BULL RACING HONDA | 69
Lando Norris | GBR | MCLAREN MERCEDES | 66
Charles Leclerc | MON | FERRARI | 52
Valtteri Bottas | FIN | MERCEDES | 47
Carlos Sainz | ESP | FERRARI | 42
Pierre Gasly | FRA | ALPHATAURI HONDA | 31
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [7]:
main.pick('Norris')

Carlaniel & the Romeos selected Lando Norris!
	87 points remaining.
Carlaniel & the Romeos | Budget: 87
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)


In [8]:
main.show_rosters()

Carlaniel & the Romeos | Budget: 87
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)

Scuderia Spaghetti | Budget: 153

3 Orange Whips | Budget: 153

Deep Fried | Budget: 153



In [9]:
main

Scuderia Spaghetti is on the clock!
****************************************
Max Verstappen | NED | RED BULL RACING HONDA | 105
Lewis Hamilton | GBR | MERCEDES | 101
Sergio Perez | MEX | RED BULL RACING HONDA | 69
Charles Leclerc | MON | FERRARI | 52
Valtteri Bottas | FIN | MERCEDES | 47
Carlos Sainz | ESP | FERRARI | 42
Pierre Gasly | FRA | ALPHATAURI HONDA | 31
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [10]:
main.pick('Hamilton')

Scuderia Spaghetti selected Lewis Hamilton!
	52 points remaining.
Scuderia Spaghetti | Budget: 52
	(2, Lewis Hamilton | GBR | MERCEDES | 101)


In [11]:
main.show_rosters()

Carlaniel & the Romeos | Budget: 87
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)

Scuderia Spaghetti | Budget: 52
	(2, Lewis Hamilton | GBR | MERCEDES | 101)

3 Orange Whips | Budget: 153

Deep Fried | Budget: 153



In [12]:
main

3 Orange Whips is on the clock!
****************************************
Max Verstappen | NED | RED BULL RACING HONDA | 105
Sergio Perez | MEX | RED BULL RACING HONDA | 69
Charles Leclerc | MON | FERRARI | 52
Valtteri Bottas | FIN | MERCEDES | 47
Carlos Sainz | ESP | FERRARI | 42
Pierre Gasly | FRA | ALPHATAURI HONDA | 31
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [13]:
main.pick('Verstappen')

3 Orange Whips selected Max Verstappen!
	48 points remaining.
3 Orange Whips | Budget: 48
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)


In [14]:
main.show_rosters()

Carlaniel & the Romeos | Budget: 87
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)

Scuderia Spaghetti | Budget: 52
	(2, Lewis Hamilton | GBR | MERCEDES | 101)

3 Orange Whips | Budget: 48
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)

Deep Fried | Budget: 153



In [15]:
main

Deep Fried is on the clock!
****************************************
Sergio Perez | MEX | RED BULL RACING HONDA | 69
Charles Leclerc | MON | FERRARI | 52
Valtteri Bottas | FIN | MERCEDES | 47
Carlos Sainz | ESP | FERRARI | 42
Pierre Gasly | FRA | ALPHATAURI HONDA | 31
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [16]:
main.pick('Bottas')

Deep Fried selected Valtteri Bottas!
	106 points remaining.
Deep Fried | Budget: 106
	(4, Valtteri Bottas | FIN | MERCEDES | 47)


In [17]:
main.show_rosters()

Carlaniel & the Romeos | Budget: 87
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)

Scuderia Spaghetti | Budget: 52
	(2, Lewis Hamilton | GBR | MERCEDES | 101)

3 Orange Whips | Budget: 48
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)

Deep Fried | Budget: 106
	(4, Valtteri Bottas | FIN | MERCEDES | 47)



## R2

In [18]:
main

Deep Fried is on the clock!
****************************************
Sergio Perez | MEX | RED BULL RACING HONDA | 69
Charles Leclerc | MON | FERRARI | 52
Carlos Sainz | ESP | FERRARI | 42
Pierre Gasly | FRA | ALPHATAURI HONDA | 31
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [19]:
main.pick('Perez')

Deep Fried selected Sergio Perez!
	37 points remaining.
Deep Fried | Budget: 37
	(4, Valtteri Bottas | FIN | MERCEDES | 47)
	(5, Sergio Perez | MEX | RED BULL RACING HONDA | 69)


In [20]:
main.show_rosters()

Carlaniel & the Romeos | Budget: 87
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)

Scuderia Spaghetti | Budget: 52
	(2, Lewis Hamilton | GBR | MERCEDES | 101)

3 Orange Whips | Budget: 48
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)

Deep Fried | Budget: 37
	(4, Valtteri Bottas | FIN | MERCEDES | 47)
	(5, Sergio Perez | MEX | RED BULL RACING HONDA | 69)



In [21]:
main

3 Orange Whips is on the clock!
****************************************
Charles Leclerc | MON | FERRARI | 52
Carlos Sainz | ESP | FERRARI | 42
Pierre Gasly | FRA | ALPHATAURI HONDA | 31
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [22]:
main.pick('Gasly')

3 Orange Whips selected Pierre Gasly!
	17 points remaining.
3 Orange Whips | Budget: 17
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)
	(6, Pierre Gasly | FRA | ALPHATAURI HONDA | 31)


In [23]:
main

Scuderia Spaghetti is on the clock!
****************************************
Charles Leclerc | MON | FERRARI | 52
Carlos Sainz | ESP | FERRARI | 42
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [24]:
main.pick('Leclerc')

Scuderia Spaghetti selected Charles Leclerc!
	0 points remaining.
Scuderia Spaghetti | Budget: 0
	(2, Lewis Hamilton | GBR | MERCEDES | 101)
	(7, Charles Leclerc | MON | FERRARI | 52)


In [25]:
main

Carlaniel & the Romeos is on the clock!
****************************************
Carlos Sainz | ESP | FERRARI | 42
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [26]:
main.pick('Sainz')

Carlaniel & the Romeos selected Carlos Sainz!
	45 points remaining.
Carlaniel & the Romeos | Budget: 45
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)
	(8, Carlos Sainz | ESP | FERRARI | 42)


## R3

In [27]:
main

Carlaniel & the Romeos is on the clock!
****************************************
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [28]:
main.pick('Ricciardo')

Carlaniel & the Romeos selected Daniel Ricciardo!
	19 points remaining.
Carlaniel & the Romeos | Budget: 19
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)
	(8, Carlos Sainz | ESP | FERRARI | 42)
	(9, Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26)


In [29]:
main

Scuderia Spaghetti is on the clock!
****************************************
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
George Russell | GBR | WILLIAMS MERCEDES | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [30]:
main.pick('Russell')

Scuderia Spaghetti selected George Russell!
	0 points remaining.
Scuderia Spaghetti | Budget: 0
	(2, Lewis Hamilton | GBR | MERCEDES | 101)
	(7, Charles Leclerc | MON | FERRARI | 52)
	(10, George Russell | GBR | WILLIAMS MERCEDES | 0)


In [31]:
main

3 Orange Whips is on the clock!
****************************************
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Fernando Alonso | ESP | ALPINE RENAULT | 13
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [32]:
main.pick('Alonso')

3 Orange Whips selected Fernando Alonso!
	4 points remaining.
3 Orange Whips | Budget: 4
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)
	(6, Pierre Gasly | FRA | ALPHATAURI HONDA | 31)
	(11, Fernando Alonso | ESP | ALPINE RENAULT | 13)


In [33]:
main

Deep Fried is on the clock!
****************************************
Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [34]:
main.pick('Vettel')

Deep Fried selected Sebastian Vettel!
	9 points remaining.
Deep Fried | Budget: 9
	(4, Valtteri Bottas | FIN | MERCEDES | 47)
	(5, Sergio Perez | MEX | RED BULL RACING HONDA | 69)
	(12, Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28)


## R4

In [35]:
main

Deep Fried is on the clock!
****************************************
Esteban Ocon | FRA | ALPINE RENAULT | 12
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [36]:
main.pick('Ocon', force=True)

Deep Fried selected Esteban Ocon!
	-3 points remaining.
Deep Fried | Budget: -3
	(4, Valtteri Bottas | FIN | MERCEDES | 47)
	(5, Sergio Perez | MEX | RED BULL RACING HONDA | 69)
	(12, Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28)
	(13, Esteban Ocon | FRA | ALPINE RENAULT | 12)


In [37]:
main

3 Orange Whips is on the clock!
****************************************
Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [38]:
main.pick('Stroll', force=True)

3 Orange Whips selected Lance Stroll!
	-5 points remaining.
3 Orange Whips | Budget: -5
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)
	(6, Pierre Gasly | FRA | ALPHATAURI HONDA | 31)
	(11, Fernando Alonso | ESP | ALPINE RENAULT | 13)
	(14, Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9)


In [39]:
main

Scuderia Spaghetti is on the clock!
****************************************
Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [40]:
main.pick('Tsunoda', force=True)

Scuderia Spaghetti selected Yuki Tsunoda!
	-8 points remaining.
Scuderia Spaghetti | Budget: -8
	(2, Lewis Hamilton | GBR | MERCEDES | 101)
	(7, Charles Leclerc | MON | FERRARI | 52)
	(10, George Russell | GBR | WILLIAMS MERCEDES | 0)
	(15, Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8)


In [41]:
main

Carlaniel & the Romeos is on the clock!
****************************************
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [42]:
main.pick('Giovinazzi')

Carlaniel & the Romeos selected Antonio Giovinazzi!
	18 points remaining.
Carlaniel & the Romeos | Budget: 18
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)
	(8, Carlos Sainz | ESP | FERRARI | 42)
	(9, Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26)
	(16, Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1)


## R5

In [43]:
main

Carlaniel & the Romeos is on the clock!
****************************************
Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [44]:
main.pick('Raikkonen')

Carlaniel & the Romeos selected Kimi Raikkonen!
	17 points remaining.
Carlaniel & the Romeos | Budget: 17
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)
	(8, Carlos Sainz | ESP | FERRARI | 42)
	(9, Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26)
	(16, Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1)
	(17, Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1)


In [45]:
main

Scuderia Spaghetti is on the clock!
****************************************
Mick Schumacher | GER | HAAS FERRARI | 0
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [46]:
main.pick('Schumacher', force=True)

Scuderia Spaghetti selected Mick Schumacher!
	-8 points remaining.
Scuderia Spaghetti | Budget: -8
	(2, Lewis Hamilton | GBR | MERCEDES | 101)
	(7, Charles Leclerc | MON | FERRARI | 52)
	(10, George Russell | GBR | WILLIAMS MERCEDES | 0)
	(15, Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8)
	(18, Mick Schumacher | GER | HAAS FERRARI | 0)


In [47]:
main

3 Orange Whips is on the clock!
****************************************
Nikita Mazepin | RAF | HAAS FERRARI | 0
Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0


In [48]:
main.pick('Latifi', force=True)

3 Orange Whips selected Nicholas Latifi!
	-5 points remaining.
3 Orange Whips | Budget: -5
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)
	(6, Pierre Gasly | FRA | ALPHATAURI HONDA | 31)
	(11, Fernando Alonso | ESP | ALPINE RENAULT | 13)
	(14, Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9)
	(19, Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0)


In [49]:
main

Deep Fried is on the clock!
****************************************
Nikita Mazepin | RAF | HAAS FERRARI | 0


In [50]:
main.pick('Mazepin', force=True)

Deep Fried selected Nikita Mazepin!
	-3 points remaining.
Deep Fried | Budget: -3
	(4, Valtteri Bottas | FIN | MERCEDES | 47)
	(5, Sergio Perez | MEX | RED BULL RACING HONDA | 69)
	(12, Sebastian Vettel | GER | ASTON MARTIN MERCEDES | 28)
	(13, Esteban Ocon | FRA | ALPINE RENAULT | 12)
	(20, Nikita Mazepin | RAF | HAAS FERRARI | 0)


# Draft Complete

## 2021 Rosters

In [51]:
main.show_rosters()

Carlaniel & the Romeos | Budget: 17
	(1, Lando Norris | GBR | MCLAREN MERCEDES | 66)
	(8, Carlos Sainz | ESP | FERRARI | 42)
	(9, Daniel Ricciardo | AUS | MCLAREN MERCEDES | 26)
	(16, Antonio Giovinazzi | ITA | ALFA ROMEO RACING FERRARI | 1)
	(17, Kimi Raikkonen | FIN | ALFA ROMEO RACING FERRARI | 1)

Scuderia Spaghetti | Budget: -8
	(2, Lewis Hamilton | GBR | MERCEDES | 101)
	(7, Charles Leclerc | MON | FERRARI | 52)
	(10, George Russell | GBR | WILLIAMS MERCEDES | 0)
	(15, Yuki Tsunoda | JPN | ALPHATAURI HONDA | 8)
	(18, Mick Schumacher | GER | HAAS FERRARI | 0)

3 Orange Whips | Budget: -5
	(3, Max Verstappen | NED | RED BULL RACING HONDA | 105)
	(6, Pierre Gasly | FRA | ALPHATAURI HONDA | 31)
	(11, Fernando Alonso | ESP | ALPINE RENAULT | 13)
	(14, Lance Stroll | CAN | ASTON MARTIN MERCEDES | 9)
	(19, Nicholas Latifi | CAN | WILLIAMS MERCEDES | 0)

Deep Fried | Budget: -3
	(4, Valtteri Bottas | FIN | MERCEDES | 47)
	(5, Sergio Perez | MEX | RED BULL RACING HONDA | 69)
	(12, Sebasti

In [52]:
main.save_rosters()

,OverallPick,TeamPick,Team,LastName,FirstName,Car,Nationality
0,1,1,Carlaniel & the Romeos,Norris,Lando,MCLAREN MERCEDES,GBR
1,2,1,Scuderia Spaghetti,Hamilton,Lewis,MERCEDES,GBR
2,3,1,3 Orange Whips,Verstappen,Max,RED BULL RACING HONDA,NED
3,4,1,Deep Fried,Bottas,Valtteri,MERCEDES,FIN
4,5,2,Deep Fried,Perez,Sergio,RED BULL RACING HONDA,MEX
5,6,2,3 Orange Whips,Gasly,Pierre,ALPHATAURI HONDA,FRA
6,7,2,Scuderia Spaghetti,Leclerc,Charles,FERRARI,MON
7,8,2,Carlaniel & the Romeos,Sainz,Carlos,FERRARI,ESP
8,9,3,Carlaniel & the Romeos,Ricciardo,Daniel,MCLAREN MERCEDES,AUS
9,10,3,Scuderia Spaghetti,Russell,George,WILLIAMS MERCEDES,GBR
